### Download Data 

In [ ]:
!wget -O m5-forecasting-accuracy.zip https://tinyurl.com/y7w5ed7w
!unzip m5-forecasting-accuracy.zip

### Import Necessary Packages

In [ ]:
import gc
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
from sklearn import preprocessing, metrics
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)


### Define Utility Functions 
* reduce memory usage 
* read data 

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
def read_data():
    print('Reading files...')
    calendar = pd.read_csv('calendar.csv')
    calendar = reduce_mem_usage(calendar)
    print('Calendar has {} rows and {} columns'.format(calendar.shape[0], calendar.shape[1]))
    sell_prices = pd.read_csv('sell_prices.csv')
    sell_prices = reduce_mem_usage(sell_prices)
    print('Sell prices has {} rows and {} columns'.format(sell_prices.shape[0], sell_prices.shape[1]))
    item_demands = pd.read_csv('sales_train_validation.csv')
    print('Sales train validation has {} rows and {} columns'.format(item_demands.shape[0], item_demands.shape[1]))
    return calendar, sell_prices, item_demands

### Show 3 Tables 
* calendar 
* sell prices 
* item demands per day 

In [ ]:
calendar, sell_prices, item_demands  = read_data()

In [ ]:
calendar.head()

In [ ]:
sell_prices.head()

In [ ]:
item_demands.head()

In [ ]:
foodproduct = item_demands[item_demands['cat_id']=='FOODS']
target_product = foodproduct['item_id'].unique()

len(target_product)

In [ ]:
foodproduct = item_demands[item_demands['cat_id']=='FOODS'][:300]
target_product = foodproduct['item_id'].unique()
item_demands = item_demands[(item_demands['item_id'].isin(target_product))]
sell_prices = sell_prices[(sell_prices['item_id'].isin(target_product))]

In [ ]:
def melt_item_demands(item_demands):
    id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
    value_vars = item_demands.columns[6:]
    item_demands = pd.melt(item_demands, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], value_vars=value_vars, var_name = 'day', value_name = 'demand')
    item_demands = reduce_mem_usage(item_demands)
    return item_demands

In [ ]:
item_demands_melt = melt_item_demands(item_demands)
item_demands_melt.head()

In [ ]:
data = pd.merge(item_demands_melt, calendar, how = 'left', left_on = ['day'], right_on = ['d'])
data.drop(['d', 'day'], inplace = True, axis = 1)
data.head()

In [ ]:
data = data.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')
data.head()

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
def transform(data):
    nan_features = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    
    for feature in nan_features: 
        data[feature].fillna('unknown', inplace = True)
        values = [[v] for v in data[feature].unique()]
        transformed = encoder.fit_transform(values)
        trans_dict = {} 
        for v,t in zip(values, transformed): 
            trans_dict[v[0]] = t[0]
        data[feature] = data[feature].apply(lambda x: int(trans_dict[x]))    
    
    return data



In [ ]:
data['event_name_1'].unique()

In [ ]:
transformed = transform(data)
transformed.head()
transformed.date.max()

In [ ]:
item_cols = ['item_id', 'dept_id', 'cat_id']
item_meta = transformed[item_cols].drop_duplicates()
item_meta.to_csv('item_meta.csv', index=False)

In [ ]:
demands_cols = ['item_id', 'date', 'demand', 'store_id', 'state_id']

demands = transformed[demands_cols]
demands = demands.rename(columns={"date":"timestamp"})

In [ ]:
demands.to_csv('demands.csv', index=False)

In [ ]:
related_time_series_cols = ['item_id','date','store_id','state_id','event_type_1','event_type_2', 'snap_CA', 'snap_TX', 'snap_WI',  'sell_price']

related_ts = transformed[related_time_series_cols]
related_ts = related_ts.rename(columns={"date":"timestamp"})


In [ ]:
related_ts.to_csv('related_ts.csv', index=False)